In [68]:
import pandas as pd 
import numpy as np
from pytictoc import TicToc

In [97]:
t = TicToc() 

# Numpy verion
# ======================================================================================
steps = 1000
last_window = np.random.normal(size=100)
lags = np.arange(5) + 1
exog = np.random.normal(size=steps)
predictions = np.full(shape=steps, fill_value=np.nan)

t.tic()
for i in range(steps):
    X = last_window[-lags].reshape(1, -1)
    if exog is not None:
        X = np.column_stack((X, exog[i, ].reshape(1, -1)))
    with warnings.catch_warnings():
        # Suppress scikit-learn warning: "X does not have valid feature names,
        # but NoOpTransformer was fitted with feature names".

        warnings.simplefilter("ignore")
        prediction = np.array([999])
        predictions[i] = prediction.ravel()[0]

    # Update `last_window` values. The first position is discarded and 
    # the new prediction is added at the end.
    last_window = np.append(last_window[1:], prediction)
t.toc()

Elapsed time is 0.018719 seconds.


In [129]:
t = TicToc() 

# Pandas verion
# ======================================================================================
steps = 1000
last_window = pd.Series(np.random.normal(size=100))
lags = np.arange(5) + 1
exog = pd.Series(np.random.normal(size=steps))
predictions = pd.Series([np.nan]*steps, dtype=float)

n_lags = len(lags)
if exog is None:
    X = pd.Series([np.nan]*lags, dtype=float)
else:
    n_exogs = exog.shape[1] if isinstance(exog, pd.DataFrame) else 1
    X = pd.Series([np.nan]*(len(lags) + n_exogs), dtype=float)

t.tic()
for i in range(steps):
    X.iloc[:n_lags] = last_window.iloc[-lags]
    if exog is not None:
        X.iloc[n_lags: ] = exog.iloc[[i, ]]
    with warnings.catch_warnings():
        # Suppress scikit-learn warning: "X does not have valid feature names,
        # but NoOpTransformer was fitted with feature names".

        warnings.simplefilter("ignore")
        prediction = np.array([999])
        predictions.iloc[i] = prediction.ravel()[0]

    # Update `last_window` values. The first position is discarded and 
    # the new prediction is added at the end.
    last_window.iloc[:-1] = last_window.iloc[1:]
    last_window[-1] = prediction
t.toc()

Elapsed time is 0.338667 seconds.


In [124]:
t = TicToc() 

# pandas verion 2
# ======================================================================================
steps = 1000
last_window = np.random.normal(size=100)
lags = np.arange(5) + 1
exog = np.random.normal(size=steps)
predictions = np.full(shape=steps, fill_value=np.nan)

last_window = last_window[-lags]
n_lags = len(lags)

if exog is None:
    X = pd.DataFrame(np.empty(shape=[1, n_lags])) # TODO: ADD THE PROPER DTYPES
else:
    n_exogs = 1 if exog.ndim == 1 else exog.shape[1]
    X = pd.DataFrame(np.empty(shape=[1, n_lags + n_exogs])) # TODO: ADD THE PROPER DTYPES

t.tic()
for i in range(steps):
    X.iloc[:, :n_lags] = last_window
    if exog is not None:
        X.iloc[:, n_lags: ] = exog[i, ]
    
    prediction = np.array([999])
    predictions[i] = prediction.ravel()[0]

    # Update `last_window` values. The first position is discarded and 
    # the new prediction is added at the end.
    last_window = np.append(last_window[1:], prediction)
t.toc()


Elapsed time is 0.112206 seconds.
